#Importamos las siguientes librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

## **NOTA:** antes de correr las siguientes líneas de código, subir el fichero kaggle.json para establecer la conexión correctamente y descargar los csv correspondientes

In [2]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'

##Descargamos los paquetes de datos de kaggle en un zip

In [3]:
!kaggle competitions download -c udea-ai4eng-20241

 94% 20.0M/21.2M [00:01<00:00, 29.9MB/s]
100% 21.2M/21.2M [00:01<00:00, 19.8MB/s]


##Descomprimimos el zip

In [4]:
!unzip udea-ai4eng-20241.zip

Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


##Guardamos las bases de datos en DataSets para poder manejarlas

In [71]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [72]:
id_train_df = train_df['ID']
id_test_df = test_df['ID']

# Mostramos la tabla

In [90]:
display(train_df)

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,7,300,4,3,3,2,1,11,1,1,0,1,5,2
1,7,249,3,1,0,2,0,10,1,0,0,1,11,1
2,6,819,4,1,4,2,1,8,1,0,0,0,8,1
3,3,14,26,2,0,3,1,4,1,0,0,1,8,0
4,7,907,1,1,2,2,1,6,1,1,0,1,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,3,144,17,4,1,1,1,8,1,0,1,1,9,2
692496,7,907,21,1,4,2,1,7,1,0,0,1,9,1
692497,0,94,4,0,3,2,1,8,1,0,1,1,9,3
692498,3,907,28,1,3,0,0,6,0,0,1,1,6,1


In [126]:
display(test_df)

,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE
0,0,914,5,5,3,2,1,10,1,0,1,1,6
1,6,13,1,1,2,1,1,8,1,0,0,1,10
2,7,449,4,0,0,2,1,9,1,0,0,1,8
3,3,228,27,0,2,0,0,7,1,0,0,0,7
4,7,14,3,1,1,1,1,8,1,0,0,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,3,14,4,1,4,0,1,7,1,1,1,1,7
296782,0,247,19,0,0,3,1,0,1,0,0,1,0
296783,7,247,4,1,4,2,1,8,1,0,0,1,0
296784,3,362,1,3,1,2,1,0,1,0,0,1,0


#Preprocesamiento de Datos

# Eliminamos las columnas que no son relevantes para nuestra prediccion

In [74]:
columns_drop = ['ID', 'FAMI_TIENEINTERNET.1','ESTU_PRIVADO_LIBERTAD']
train_df = train_df.drop(columns_drop, axis = 1)
test_df = test_df.drop(columns_drop, axis = 1)

# Relleno las filas que tienen datos vacios

In [75]:
# Evalúa si hay valores nulos en el dataset y los imputa
def impute_missing_values(df):
    imputer = SimpleImputer(strategy='most_frequent')
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
    return df_imputed

train_df = impute_missing_values(train_df)
test_df = impute_missing_values(test_df)

#Las carreras que tengan menos de 5 personas inscritas en esta quedaran como 1 sola categoria llamada "otros"

In [43]:
# juntar varias carreras con menos de 5 datos en una sola en train_df ---------------------------------

#contar la cantidad de veces que esta cada programa academico
programa_academico_counts = train_df['ESTU_PRGM_ACADEMICO'].value_counts()

#definimos el umbral que consideramos como categoria rara
umbral = 5

#Identificacmos los programas academicos que considderamos como datos atipicos
programa_academicos_atipico = programa_academico_counts[programa_academico_counts < umbral].index

#Reemplazamos programas academicos atipicos como "otros"
train_df['ESTU_PRGM_ACADEMICO'] = train_df['ESTU_PRGM_ACADEMICO'].replace(programa_academicos_atipico, 'OTROS')

#Verificamos cambios
print(train_df['ESTU_PRGM_ACADEMICO'].value_counts())

ESTU_PRGM_ACADEMICO
DERECHO                                            53244
ADMINISTRACION DE EMPRESAS                         51902
CONTADURIA PUBLICA                                 39664
PSICOLOGIA                                         31932
INGENIERIA INDUSTRIAL                              28481
                                                   ...  
LICENCIATURA EN  LITERATURA Y LENGUA CASTELLANA        5
MICROBIOLOGÍA Y BIOANÁLISIS                            5
PROFESIONAL SEGURIDAD Y SALUD EN EL TRABAJO            5
INGENIERÍA BIOMÉDICA                                   5
GESTIÓN CULTURAL                                       5
Name: count, Length: 890, dtype: int64


In [44]:
# juntar varias carreras con menos de 5 datos en una sola en test_df---------------------------------

#contar la cantidad de veces que esta cada programa academico
programa_academico_counts = test_df['ESTU_PRGM_ACADEMICO'].value_counts()

#definimos el umbral que consideramos como categoria rara
umbral = 5

#Identificacmos los programas academicos que considderamos como datos atipicos
programa_academicos_atipico = programa_academico_counts[programa_academico_counts < umbral].index

#Reemplazamos programas academicos atipicos como "otros"
test_df['ESTU_PRGM_ACADEMICO'] = test_df['ESTU_PRGM_ACADEMICO'].replace(programa_academicos_atipico, 'OTROS')

#Verificamos cambios
print(test_df['ESTU_PRGM_ACADEMICO'].value_counts())

ESTU_PRGM_ACADEMICO
DERECHO                                               22984
ADMINISTRACION DE EMPRESAS                            22298
CONTADURIA PUBLICA                                    16861
PSICOLOGIA                                            13818
INGENIERIA INDUSTRIAL                                 12516
                                                      ...  
INGENIERÍA AGRONOMICA                                     5
PROFESIONAL EN DISEÑO DE VESTUARIO                        5
ADMINISTRACIÓN AGROINDUSTRIAL                             5
ESTUDIOS LITERARIOS Y EDICION                             5
LICENCIATURA EN FILOSOFIA, ETICA Y VALORES HUMANOS        5
Name: count, Length: 852, dtype: int64


# XGBoost

In [119]:
# Selección de columnas categóricas
columnas_categoricas =[
    'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
    'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
    'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
    'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE','RENDIMIENTO_GLOBAL'
]

In [120]:
# Aplicar LabelEncoder a cada columna categórica
label_encoder = LabelEncoder()
for col in columnas_categoricas:
    train_df[col] = label_encoder.fit_transform(train_df[col])

In [121]:
# Separa las características (X) y etiqueta (y) en los datos de entrenamiento
X_train = train_df.drop('RENDIMIENTO_GLOBAL', axis=1)
y_train = train_df['RENDIMIENTO_GLOBAL']
# Establece las características en los datos de prueba
X_test = test_df

In [122]:
# Codifica la etiqueta categórica en y_train
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [123]:
# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [124]:
# Inicializar y entrenar el modelo XGBoost
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [125]:
#----------------------------------------------------------------------------------------------

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

#---------------------------------------------------------------------------------------------

Accuracy: 43.52%
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.63      0.59     35165
           1       0.46      0.57      0.51     34573
           2       0.33      0.27      0.30     34259
           3       0.33      0.26      0.30     34503

    accuracy                           0.44    138500
   macro avg       0.42      0.43      0.42    138500
weighted avg       0.42      0.44      0.42    138500

Confusion Matrix:
[[22256  3007  6466  3436]
 [ 2824 19685  4763  7301]
 [10080  7554  9205  7420]
 [ 5199 12641  7532  9131]]


In [134]:
# Selección de columnas categóricas
columnas_categoricas =[
    'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
    'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
    'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
    'FAMI_TIENECOMPUTADOR', 'FAMI_EDUCACIONMADRE'
]
for col in columnas_categoricas:
    test_df[col] = label_encoder.fit_transform(test_df[col])

X_test = test_df

# Realizar predicciones
y_pred = model.predict(X_test)

In [136]:
#Concateno el ID con el redimiento global categorico
id = pd.DataFrame({'ID': id_test_df})
results = pd.DataFrame({'RENDIMIENTO_GLOBAL': y_pred})
df_concat = pd.concat([id, results], axis=1)

In [139]:
#convierto las variables numericas en categoricas
mapeo = {0: "bajo", 1: "medio-bajo", 2: "medio-alto", 3: "alto"}
df_concat['RENDIMIENTO_GLOBAL']= df_concat['RENDIMIENTO_GLOBAL'].replace(mapeo)

In [140]:
print(df_concat)

            ID RENDIMIENTO_GLOBAL
0       550236         medio-alto
1        98545               alto
2       499179               bajo
3       782980         medio-bajo
4       785185         medio-bajo
...        ...                ...
296781  496981               alto
296782  209415         medio-alto
296783  239074         medio-alto
296784  963852               bajo
296785  792650               bajo

[296786 rows x 2 columns]


In [141]:
df_concat.to_csv('submission.csv', index=False)

In [142]:
!kaggle competitions submit -c udea-ai4eng-20241 -f submission.csv -m "Submission1"

100% 4.21M/4.21M [00:01<00:00, 2.97MB/s]
Successfully submitted to UDEA/ai4eng 20241 - Pruebas Saber Pro Colombia